In [1]:
import pixiedust
import pandas as pd
import numpy as np

from src.my_aws import S3

Pixiedust database opened successfully


In [2]:
KEY_FINAL = 'Final_Data.csv'
BUCKET = 'movie-torrents'

s3_obj = S3()
df = s3_obj.get_data(KEY_FINAL, BUCKET)

In [3]:
display(df)

Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Production,Rated,Released,Runtime,Title,Type,Writer,imdbID,imdbRating,imdbVotes,Kat_Count,Pirate_Count,Extra_Count,Torrentz_Count,Torrentz_Ver_Count,Zoogle_Ver_Count,ProductionBudget,DomesticBudget,WorldGross,Total_Torrents,ProductionBudget_Log,DomesticBudget_Log,DVD_Month,DVD_Year,Release_Month,Release_Year,Rated_Trans,Production_Trans,Prod_Count,Prod_Size,Genre_Drama,Genre_Comedy,Genre_Action,Genre_Adventure,Genre_Romance,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Mystery,Genre_Fantasy,Genre_Family,Genre_Other
"Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang",Won 3 Oscars. Another 85 wins & 128 nominations.,749700000.0,"UK, USA",2010-04-22,James Cameron,"Action, Adventure, Fantasy","English, Spanish",83.0,20th Century Fox,PG-13,2009-12-18,162.0,Avatar,movie,James Cameron,tt0499549,7.80000019073,944150.0,509,371,140,961,21,118.0,425000000,760507625,2783918982,2120.0,19.8675997269,20.4494966959,4.0,2010.0,12,2009,PG-13,20th century fox,319,large,0,0,1,1,0,0,0,0,0,0,1,0,0
"Hugh Jackman, Halle Berry, Ian McKellen, Patrick Stewart",7 wins & 37 nominations.,234300000.0,"Canada, USA, UK",2006-10-03,Brett Ratner,"Action, Adventure, Fantasy",English,58.0,20th Century Fox,PG-13,2006-05-26,104.0,X-Men: The Last Stand,movie,"Simon Kinberg, Zak Penn",tt0376994,6.69999980927,412975.0,166,0,60,226,11,53.0,210000000,234362462,459260946,516.0,19.1626180887,19.2723794578,10.0,2006.0,5,2006,PG-13,20th century fox,319,large,0,0,1,1,0,0,0,0,0,0,1,0,0
"Hugh Jackman, James McAvoy, Michael Fassbender, Jennifer Lawrence",Nominated for 1 Oscar. Another 15 wins & 47 nominations.,199305306.0,"USA, UK, Canada",2014-10-14,Bryan Singer,"Action, Adventure, Sci-Fi","English, Vietnamese, French",74.0,20th Century Fox,PG-13,2014-05-23,132.0,X-Men: Days of Future Past,movie,"Simon Kinberg (screenplay), Jane Goldman (story by), Simon Kinberg (story by), Matthew Vaughn (story by)",tt1877832,8.0,561421.0,231,251,497,905,95,113.0,200000000,233921534,747862775,2092.0,19.1138279245,19.2704962923,10.0,2014.0,5,2014,PG-13,20th century fox,319,large,0,0,1,1,0,0,0,0,1,0,0,0,0
"James McAvoy, Michael Fassbender, Jennifer Lawrence, Nicholas Hoult",13 nominations.,135729385.0,USA,2016-10-04,Bryan Singer,"Action, Adventure, Sci-Fi","English, Polish, German, Arabic, Egyptian (Ancient)",52.0,20th Century Fox,PG-13,2016-05-27,144.0,X-Men: Apocalypse,movie,"Simon Kinberg (screenplay), Bryan Singer (story by), Simon Kinberg (story by), Michael Dougherty (story by), Dan Harris (story by)",tt3385516,7.0,290362.0,150,26,66,302,0,130.0,178000000,155442489,542742489,674.0,18.9972941083,18.8617863755,10.0,2016.0,5,2016,PG-13,20th century fox,319,large,0,0,1,1,0,0,0,0,1,0,0,0,0
"Andy Serkis, Jason Clarke, Gary Oldman, Keri Russell",Nominated for 1 Oscar. Another 16 wins & 47 nominations.,183278877.0,"USA, UK, Canada",2014-12-02,Matt Reeves,"Action, Adventure, Drama","English, American Sign Language",79.0,20th Century Fox,PG-13,2014-07-11,130.0,Dawn of the Planet of the Apes,movie,"Mark Bomback, Rick Jaffa, Amanda Silver, Rick Jaffa (characters), Amanda Silver (characters), Pierre Boulle (novel)",tt2103281,7.59999990463,351165.0,477,182,184,524,46,66.0,170000000,208545589,710644566,1479.0,18.951308995,19.1556682276,12.0,2014.0,7,2014,PG-13,20th century fox,319,large,1,0,1,1,0,0,0,0,0,0,0,0,0
"Liam Hemsworth, Jeff Goldblum, Jessie T. Usher, Bill Pullman",2 wins & 14 nominations.,103137295.0,USA,2016-10-18,Roland Emmerich,"Action, Adventure, Sci-Fi","English, Mandarin",32.0,20th Century Fox,PG-13,2016-06-24,120.0,Independence Day: Resurgence,movie,"Nicolas Wright (screenplay), James A. Woods (screenplay), Dean Devlin (screenplay), Roland Emmerich (screenplay), James Vanderbilt (screenplay), Dean Devlin (story by), Roland Emmerich (story by), Nicolas Wright (story by), James A. Woods (story by), Dean Devlin (based on characters created by), Roland Emmerich (based on characters created by)",tt1

In [4]:
display(df)

In [5]:
_min = 50
_max = 200
res = len(df) - len(df[(df['Runtime'] >= _min) & (df['Runtime'] <= _max)])
print('{0} records dropped if dataset is limited to a runtime length between {1}-{2} minutes'.format(res, _min, _max))

71 records dropped if dataset is limited to a runtime length between 50-200 minutes


In [6]:
# Drop records stated above (runtime length)
df = df[(df['Runtime'] >= _min) & (df['Runtime'] <= _max)]

### - Remove movies where runtime is less than 50 (and greater than 200) minutes?
___

In [7]:
display(df)

In [8]:
ct = len(df[df['DomesticBudget'] <= 0])
print('''{0} records dropped due to Domestic Budget of 0 or less -
         majority of these films are low torrent count/indie style films'''.format(ct))

df = df[df['DomesticBudget'] > 0]

df['ProductionBudget_Log'] = np.log(df['ProductionBudget'])
df['DomesticBudget_Log'] = np.log(df['DomesticBudget'])

218 records dropped due to Domestic Budget of 0 or less -
         majority of these films are low torrent count/indie style films


In [9]:
display(df)

### - Try taking log-transform of * Budget values
___

In [10]:
# Extract DVD release month and year
df['DVD_Month'] = pd.to_numeric(pd.to_datetime(df['DVD']).apply(lambda x: x.month), errors='coerce')
df['DVD_Year'] = pd.to_numeric(pd.to_datetime(df['DVD']).apply(lambda x: x.year), errors='coerce')

# Extract theaters release month and year
df['Release_Month'] = pd.to_numeric(pd.to_datetime(df['Released']).apply(lambda x: x.month), errors='coerce')
df['Release_Year'] = pd.to_numeric(pd.to_datetime(df['Released']).apply(lambda x: x.year), errors='coerce')

df.dropna(subset=['DVD_Month','DVD_Year','Release_Month','Release_Year'], inplace=True)

In [11]:
display(df)

### - Change bin count to 4 for seasonal (spring, summer, fall, winter) and to 12 for monthly
___

In [12]:
display(df)

### - Change bin count to 4 for seasonal (spring, summer, fall, winter) and to 12 for monthly
---

# Transform Genres to Select Groups

In [13]:
df['Rated_Trans'] = df['Rated'].copy()
df['Rated_Trans'].fillna(value='NR', inplace=True)
df.loc[df['Rated_Trans'].isin(['NOT RATED','UNRATED']), 'Rated_Trans'] = 'NR'
df.loc[df['Rated_Trans'].isin(['GP','TV-PG','M','M/PG','APPROVED','PASSED']), 'Rated_Trans'] = 'PG'
df.loc[df['Rated_Trans'].isin(['TV-G']), 'Rated_Trans'] = 'G'
df.loc[df['Rated_Trans'].isin(['TV-14','NC-17']), 'Rated_Trans'] = 'PG-13'
df.loc[df['Rated_Trans'].isin(['X']), 'Rated_Trans'] = 'R'
df['Rated_Trans'].unique().tolist()

['PG-13', 'PG', 'G', 'R', 'NR']

In [14]:
display(df)

# Group by Size of Production Company

In [15]:
df['Production_Trans'] = df['Production']
df['Production_Trans'].fillna('', inplace=True)
df['Production_Trans'] = df['Production'].str.lower().apply(lambda x: str(str(x).split('/')[0]))

In [16]:
df.loc[df['Production_Trans'].str.contains('century fox'), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].str.contains('20th century'), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].isin(['fox']), 'Production_Trans'] = '20th century fox'
df.loc[df['Production_Trans'].isin(['foxfaith']), 'Production_Trans'] = 'fox faith films'
df.loc[df['Production_Trans'].str.contains('lions'), 'Production_Trans'] = 'lionsgate'
df.loc[df['Production_Trans'].str.contains('dream'), 'Production_Trans'] = 'dreamworks'
df.loc[df['Production_Trans'].str.contains('destination'), 'Production_Trans'] = 'destination films'
df.loc[df['Production_Trans'].str.contains('yari'), 'Production_Trans'] = 'yari film group'
df.loc[df['Production_Trans'].str.contains('yash'), 'Production_Trans'] = 'yash raj films'

names = ['warner','mca','universal','paramount','sony','columbia','new line','mgm','miramax',
         'disney','buena vista','united artists','weinstein','touchstone','fox searchlight','summit',
         'relativity','magnolia','tristar','anchor bay','gramercy','open road','ifc','hollywood', 'district']

for name in names:
    df.loc[df['Production_Trans'].str.contains(name), 'Production_Trans'] = name

In [17]:
prod = df.groupby('Production_Trans').count().sort_values('Actors',ascending=False)['Actors'].reset_index()
prod.columns = [['Production_Trans', 'Prod_Count']]
# xtreme > 400
# large <= 400 x > 300
# medium <= 300 x > 100
# small <= 100 x > 20
# tiny <= 20

a = 400
b = 300
c = 100
d = 20

prod['Prod_Size'] = 'tiny'
prod.loc[prod['Prod_Count'] > a, 'Prod_Size'] = 'xtreme'
prod.loc[(prod['Prod_Count'] <= a) & (prod['Prod_Count'] > b), 'Prod_Size'] = 'large'
prod.loc[(prod['Prod_Count'] <= b) & (prod['Prod_Count'] > c), 'Prod_Size'] = 'medium'
prod.loc[(prod['Prod_Count'] <= c) & (prod['Prod_Count'] > d), 'Prod_Size'] = 'small'

In [18]:
# Bring results into main dataframe
df = pd.merge(df, prod)

# Make Genres into individual features with cutoff 

In [ ]:
# split strings into a numpy array
def split_to_array(ser):
    split_array = np.array(ser.strip().split(','))
    return pd.Series(split_array).apply(lambda x: x.strip())

In [32]:
# turn numpy array into count of occurances
genres = df['Genre'].apply(split_to_array)
genres = pd.Series(genres.values.ravel()).dropna()
genres = genres.value_counts().sort_values(ascending=False)

# convert series to dataframe
genre_ser = genres.reset_index()
genre_ser.columns = ['Genre', 'Count']

In [34]:
# Flag top genres that film is listed under having
top_genres = genre_ser['Genre'][:12].tolist()
other_genres = genre_ser['Genre'][12:].tolist()

for gen in top_genres:
    df[gen] = 0
    df.loc[df['Genre'].str.contains(gen), gen] = 1

# For remaining genres - flage under `Other`
df['Genre_Other'] = 0
for o_gen in other_genres:
    df.loc[df['Genre'].str.contains(o_gen), 'Genre_Other'] = 1
    
# Rename columns to better explain purpose 
rename_genres = {x: 'Genre_{0}'.format(x) for x in top_genres}

df = df.rename(columns = rename_genres)

# Country

In [43]:
# turn numpy array into count of occurances
countries = df['Country'].apply(split_to_array)
countries = pd.Series(countries.values.ravel()).dropna()
countries = countries.value_counts().sort_values(ascending=False)

# convert series to dataframe for plotting
countries = countries.reset_index()
countries.columns = ['Country', 'Count']

# Flag top countries that film is listed under
top_ctrys = countries['Country'][:5].tolist()
other_ctrys = countries['Country'][5:].tolist()

for ctry in top_ctrys:
    df[ctry] = 0
    df.loc[df['Country'].str.contains(ctry), ctry] = 1

# For remaining countries - flag under `Other`
df['Country_Other'] = 0
for o_ctry in other_ctrys:
    df.loc[df['Country'].str.contains(o_ctry), 'Country_Other'] = 1
    
# Rename columns to better explain purpose 
rename_countries = {x: 'Country_{0}'.format(x) for x in top_ctrys}

df = df.rename(columns = rename_countries)

In [50]:
# turn numpy array into count of occurances
_dir = df['Director'].apply(split_to_array)
_dir = pd.Series(_dir.values.ravel()).dropna()
_dir = _dir.value_counts().sort_values(ascending=False)

# convert series to dataframe for plotting
_dir = _dir.reset_index()
_dir.columns = ['Director', 'Count']
_dir

,Director,Count
0,Steven Spielberg,26
1,Woody Allen,21
2,Clint Eastwood,20
3,Martin Scorsese,19
4,Steven Soderbergh,16
5,Ridley Scott,16
6,Spike Lee,14
7,Robert Zemeckis,13
8,Barry Levinson,13
9,Oliver Stone,13


In [48]:
df.columns

Index(['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director', 'Genre',
       'Language', 'Metascore', 'Production', 'Rated', 'Released', 'Runtime',
       'Title', 'Type', 'Writer', 'imdbID', 'imdbRating', 'imdbVotes',
       'Kat_Count', 'Pirate_Count', 'Extra_Count', 'Torrentz_Count',
       'Torrentz_Ver_Count', 'Zoogle_Ver_Count', 'ProductionBudget',
       'DomesticBudget', 'WorldGross', 'Total_Torrents',
       'ProductionBudget_Log', 'DomesticBudget_Log', 'DVD_Month', 'DVD_Year',
       'Release_Month', 'Release_Year', 'Rated_Trans', 'Production_Trans',
       'Prod_Count', 'Prod_Size', 'Genre_Drama', 'Genre_Comedy',
       'Genre_Action', 'Genre_Adventure', 'Genre_Romance', 'Genre_Crime',
       'Genre_Thriller', 'Genre_Horror', 'Genre_Sci-Fi', 'Genre_Mystery',
       'Genre_Fantasy', 'Genre_Family', 'Genre_Other', 'Genre_Drama',
       'Genre_Comedy', 'Genre_Action', 'Genre_Adventure', 'Genre_Romance',
       'Genre_Crime', 'Genre_Thriller', 'Genre_Horror', 'Genre_Sci

In [ ]:
df = df[['Director', 'Genre',
       'Language', 'Metascore', 'Production', 'Rated', 'Released', 'Runtime',
       'Title', 'Type', 'Writer', 'imdbID', 'imdbRating', 'imdbVotes',
       'Kat_Count', 'Pirate_Count', 'Extra_Count', 'Torrentz_Count',
       'Torrentz_Ver_Count', 'Zoogle_Ver_Count', 'ProductionBudget',
       'DomesticBudget', 'WorldGross', 'Total_Torrents',
       'ProductionBudget_Log', 'DomesticBudget_Log', 'DVD_Month', 'DVD_Year',
       'Release_Month', 'Release_Year', 'Rated_Trans', 'Production_Trans',
       'Prod_Count', 'Prod_Size', 'Genre_Drama', 'Genre_Comedy',
       'Genre_Action', 'Genre_Adventure', 'Genre_Romance', 'Genre_Crime',
       'Genre_Thriller', 'Genre_Horror', 'Genre_Sci-Fi', 'Genre_Mystery',
       'Genre_Fantasy', 'Genre_Family', 'Genre_Other', 'Genre_Drama',
       'Genre_Comedy', 'Genre_Action', 'Genre_Adventure', 'Genre_Romance',
       'Genre_Crime', 'Genre_Thriller', 'Genre_Horror', 'Genre_Sci-Fi',
       'Genre_Mystery', 'Genre_Fantasy', 'Genre_Family', 'Genre_Other',
       'Country_USA', 'Country_UK', 'Country_Germany', 'Country_Canada',
       'Country_France', 'Country_Other']]